In [ ]:
#creating pivot table: FID vertically


import pandas as pd
import numpy as np
from datetime import datetime

def create_stacked_timeseries(excel_file):
    """
    Create time series for parameters with all FIDs stacked vertically
    """
    # Read the Excel file
    print("Reading Excel file...")
    df = pd.read_excel(excel_file, index_col=0)
    print(f"Number of columns: {len(df.columns)}")
    print(f"Number of rows: {len(df)}")

    # Parameters mapping (single letters to full names)
    parameters = {
        'm': 'monitoring',
        't': 'temperature',
        'b': 'bc',
        'c': 'cmaqPM',
        'd': 'dustpm',
        'a': 'aod',
        'r': 'rain_merra',
        'h': 'RH_merra',
        'w': 'wind_merra',
        'e': 'rad_merra'  # 'e' might be for energy/radiation
    }

    print("Processing data...")
    all_data_list = []
    processed_dates = []

    # Get the date range (assuming one year of daily data)
    dates = pd.date_range(start='2016-01-01', end='2016-12-31', freq='D')

    # Process each row (FID)
    total_rows = len(df)
    for count, (idx, row) in enumerate(df.iterrows()):
        if count % 100 == 0:
            print(f"Processing FID {count+1} of {total_rows}...")

        # Process each parameter
        for param_prefix, param_name in parameters.items():
            # Get columns for this parameter (including those with dots)
            param_cols = sorted([col for col in df.columns if col.startswith(param_prefix)],
                             key=lambda x: int(x.split('.')[-1]) if '.' in x else 0)

            # Skip if no columns found for this parameter
            if not param_cols:
                continue

            # Create data for each column
            for col_idx, col in enumerate(param_cols):
                if col_idx < len(dates):  # Ensure we don't exceed the date range
                    data_row = {
                        'FID': idx,
                        'date': dates[col_idx],
                        'parameter': param_name,
                        'value': row[col]
                    }
                    all_data_list.append(data_row)

    print(f"Total number of data points collected: {len(all_data_list)}")

    if not all_data_list:
        raise ValueError("No data was collected. Please check the input file and parameters.")

    # Create DataFrame from the collected data
    print("Creating final DataFrame...")
    final_df = pd.DataFrame(all_data_list)

    # Sort by FID, date, and parameter
    print("Sorting data...")
    final_df = final_df.sort_values(['FID', 'date', 'parameter'])

    # Save to CSV
    print("Saving stacked format...")
    final_df.to_csv('stacked_timeseries.csv', index=False)

    # Create pivot table version
    print("Creating pivot table...")
    pivot_df = final_df.pivot_table(
        index=['FID', 'date'],
        columns='parameter',
        values='value',
        aggfunc='first'
    ).reset_index()

    # Sort by FID and date
    pivot_df = pivot_df.sort_values(['FID', 'date'])

    # Save the pivot version to CSV
    print("Saving pivot format...")
    pivot_df.to_csv('pivot_timeseries.csv', index=False)

    print("Processing complete!")
    print(f"Final stacked shape: {final_df.shape}")
    print(f"Final pivot shape: {pivot_df.shape}")

    return final_df, pivot_df

# Usage
if __name__ == "__main__":
    excel_file = "model_parameters.xlsx"
    try:
        stacked_df, pivot_df = create_stacked_timeseries(excel_file)

        # Print sample of the results
        print("\nSample of stacked format:")
        print(stacked_df.head(10))

        print("\nSample of pivot format:")
        print(pivot_df.head(10))

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
#Verification Plot


import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('pivot_timeseries.csv')

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter for FID 11
fid_11 = df[df['FID'] == 3100]

# Create the plot
plt.figure(figsize=(15, 8))

# Plot each parameter
plt.plot(fid_11['date'], fid_11['aod'], label='AOD', color='purple')
plt.plot(fid_11['date'], fid_11['dustpm'], label='Dust PM', color='green')
plt.plot(fid_11['date'], fid_11['cmaqPM'], label='CMAQ PM', color='orange')

# Customize the plot
plt.title('Timeseries Plot for FID 11 (2016)', fontsize=14, pad=20)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Value', fontsize=12)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add grid
plt.grid(True, linestyle='--', alpha=0.7)

# Add legend
plt.legend()

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Save the plot
plt.savefig('timeseries_FID_11.png')
plt.show()

# Print some summary statistics
print("\nSummary Statistics for FID 11:")
print(fid_11[['aod', 'dustpm', 'cmaqPM']].describe())